# OpenCV

**"A library of programming functions mainly aimed at real-time computer vision"**



## Pregled
* Osnovne operacije sa slikom
* Crtanje slike
* Prostori boja
* Jednostavne transformacije
* Thresholdovanje
* Konvolucije
* Morfološke transformacije
* Bounding box
* Rad sa videom, background subtraction
* Rad sa kamerom, color finder

## Uvod
* OpenCV je jedna od najkorišćenijih biblioteka za *computer vision* - pisana je u C++ ali su naknadno napisani interfejsi za još nekoliko jezika, uključujući i Python
* Iako sadrži neke tehnike mašinskog učenja, OpenCV nam je uglavnom koristan u procesu pretprocesiranja i pripreme podataka, kao i za implementaciju okolnih funkcionalnosti koje uz dobar ML model mogu da čine funkcionalnu aplikaciju
* **Napomena**: Funkcija za prikaz slike (`cv2.imshow`) ne funkcioniše u Google Colab, pa koristimo posebnu funkciju `google.colab.patches.cv2_imshow` - u slučaju pokretanja ovog koda kao py kod preći na `cv2.imshow`
* **Napomena**: Komunikacija sa web kamerom ne funkcioniše u Google Colab, kopirati taj blok koda u `.py` fajl i pokrenuti lokalno

## Import

In [0]:
# TODO

## Osnovne operacije sa slikom

In [0]:
# TODO

## Crtanje slike

In [0]:
# TODO

## Prostori boja

In [0]:
# TODO

## Jednostavne transformacije

In [0]:
# TODO

In [0]:
# TODO

## Thresholdovanje
[Dokumentacija svih metoda thresholdovanja](https://docs.opencv.org/3.0-alpha/modules/imgproc/doc/miscellaneous_transformations.html?highlight=thresh_tozero_inv#threshold
)

In [0]:
# TODO

In [0]:
# TODO

## Konvolucije
Primena konvolucije na sliku sa fiksnim kernelima, o čemu je bilo reči u toku priče o CNN


[Neki primeri](https://en.wikipedia.org/wiki/Kernel_(image_processing))

In [0]:
# TODO

In [0]:
# TODO

## Morfološke transformacije

In [0]:
# TODO

## Bounding box

In [0]:
# TODO

## Rad sa videom, background subtraction
- **Pokrenuti u lokalu!**

In [0]:
# TODO

## Rad sa kamerom, color finder
- **Pokrenuti u lokalu!**

In [0]:
# TODO

## Resursi

- [Zvanična OpenCV dokumentacija](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html)
- Sama dokumentacija funkcija nije toliko dobra, ali postoji veliki broj gotovih tutoriala koji uvode neke od mnogobrojnih algoritama/tehnika prisutnih u OpenCV
- Neke od njih koje nismo stigli danas da obradimo:
  - [Konture](https://docs.opencv.org/3.3.1/d4/d73/tutorial_py_contours_begin.html)
  - [Pronalaženje linija](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html#hough-lines)
  - [Izdvajanje prednjeg plana (GrabCut)](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_grabcut/py_grabcut.html#grabcut)
  - [Segmentacija slike (watershed)](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_watershed/py_watershed.html)
- [Odličan blog za Computer Vision sa dosta tutoriala i primera: pyimagesearch.com](https://www.pyimagesearch.com/)